In [1]:
%%capture
!pip install sentencepiece
!pip install transformers

In [2]:
#DO ALL NECESSARY IMPORTS
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import CamembertConfig, CamembertModel, CamembertTokenizer, CamembertTokenizer, CamembertForSequenceClassification
from transformers import BertModel, BertTokenizer
import torch


In [3]:
#use bert embedding but using camembert as model
model_name = 'camembert-base'
camembert_model = CamembertModel.from_pretrained(model_name)
tokenizer = CamembertTokenizer.from_pretrained(model_name, revision="main", sentencepiece_model="/usr/local/lib/python3.10/dist-packages")


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

def get_bert_embedding(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

def max_pool_embeddings(embeddings):
    return np.max(embeddings, axis=0)

def final_camembert(df):
    df['camembert_embedding'] =  df['sentence'].apply(lambda x: get_bert_embedding(x, camembert_model, tokenizer))
    df['cam_pooled_embedding'] = df['camembert_embedding'].apply(lambda x: max_pool_embeddings(x))
    return df

In [5]:
import pandas as pd
test = pd.read_csv('https://raw.githubusercontent.com/eperroud/DataScienceProject/70e3027b81808771ac12a7aae2238bce06b51f8d/data/final_test.csv')
training = pd.read_csv('https://raw.githubusercontent.com/eperroud/DataScienceProject/b11a6530812e1311cfe11e2c7a6a20e1b544b796/data/final_training.csv')

In [6]:
training = final_camembert(training)

In [7]:
test = final_camembert(test)

In [8]:
test.head()

,id,sentence,sentence_sp,tokens,tokens_no_stop,token_count_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram,cognate_count,camembert_embedding,cam_pooled_embedding
0,0,Nous dûmes nous excuser des propos que nous eû...,Nous dûmes nous excuser des propos que nous eû...,"['Nous', 'dûmes', 'nous', 'excuser', 'des', 'p...","['dûmes', 'excuser', 'propos', 'eûmes', 'prono...",5,10,3,1,0,0,"['nous', 'dûmes', 'nous', 'excuser', 'des', 'p...",3.192917,2.331769,0,"[[0.047271907, 0.030453198, -0.022989253, 0.10...","[0.047271907, 0.030453198, -0.022989253, 0.109..."
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,Vous ne pouvez pas savoir le plaisir que j'ai ...,"['Vous', 'ne', 'pouvez', 'pas', 'savoir', 'le'...","['pouvez', 'savoir', 'plaisir', 'recevoir', 'b...",7,16,4,1,2,2,"['vous', 'ne', 'pouvez', 'pas', 'savoir', 'le'...",12.660056,8.902162,0,"[[-0.0065091653, -0.11096406, -0.0380778, 0.06...","[-0.0065091653, -0.11096406, -0.0380778, 0.065..."
2,2,"Et, paradoxalement, boire froid n'est pas la b...","Et, paradoxalement, boire froid n'est pas la b...","['Et', ',', 'paradoxalement', ',', 'boire', 'f...","[',', 'paradoxalement', ',', 'boire', 'froid',...",8,13,1,1,2,3,"['et,', 'paradoxalement,', 'boire', 'froid', ""...",8.244312,5.840872,0,"[[-0.035896078, -0.04058774, 0.08143554, 0.039...","[-0.035896078, -0.04058774, 0.08143554, 0.0398..."
3,3,"Ce n'est pas étonnant, car c'est une saison my...","Ce n'est pas étonnant, car c'est une saison my...","['Ce', ""n'"", 'est', 'pas', 'étonnant', ',', 'c...","['étonnant', ',', 'saison', 'mystérieuse']",4,12,0,1,2,2,"['ce', ""n'est"", 'pas', 'étonnant,', 'car', ""c'...",1.943283,1.444063,0,"[[0.034818918, -0.037731417, -0.055650946, 0.0...","[0.034818918, -0.037731417, -0.055650946, 0.08..."
4,4,"Le corps de Golo lui-même, d'une essence aussi...","Le corps de Golo lui-même, d'une essence aussi...","['Le', 'corps', 'de', 'Golo', 'lui-même', ',',...","['corps', 'Golo', ',', 'essence', 'surnaturell...",40,83,9,13,5,8,"['le', 'corps', 'de', 'golo', 'lui-même,', ""d'...",15.581025,10.953801,0,"[[0.011139873, 0.057788055, -0.029189765, 0.12...","[0.011139873, 0.057788055, -0.029189765, 0.120..."


In [9]:
training.head()

,sentence,difficulty,encoded_diff,sentence_sp,tokens,tokens_no_stop,token_count_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram,cognate_count,camembert_embedding,cam_pooled_embedding
0,Les coûts kilométriques réels peuvent diverger...,C1,4,Les coûts kilométriques réels peuvent diverger...,"['Les', 'coûts', 'kilométriques', 'réels', 'pe...","['coûts', 'kilométriques', 'réels', 'diverger'...",27,45,4,14,3,2,"['les', 'coûts', 'kilométriques', 'réels', 'pe...",27.043962,18.477813,1,"[[0.04393054, -0.012339459, 0.03234782, 0.1047...","[0.04393054, -0.012339459, 0.03234782, 0.10470..."
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,0,"Le bleu, c'est ma couleur préférée mais je n'a...","['Le', 'bleu', ',', ""c'"", 'est', 'ma', 'couleu...","['bleu', ',', 'couleur', 'préférée', 'aime', '...",7,16,1,3,1,2,"['le', 'bleu,', ""c'est"", 'ma', 'couleur', 'pré...",6.295061,4.283070,1,"[[0.031562533, -0.064943954, -0.062135864, 0.0...","[0.031562533, -0.064943954, -0.062135864, 0.05..."
2,Le test de niveau en français est sur le site ...,A1,0,Le test de niveau en français est sur le site ...,"['Le', 'test', 'de', 'niveau', 'en', 'français...","['test', 'niveau', 'français', 'site', 'Intern...",7,15,1,4,1,0,"['le', 'test', 'de', 'niveau', 'en', 'français...",35.379919,23.504444,0,"[[0.017271552, -0.05883773, -0.020093849, -0.0...","[0.017271552, -0.05883773, -0.020093849, -0.04..."
3,Est-ce que ton mari est aussi de Boston?,A1,0,Est-ce que ton mari est aussi de Boston?,"['Est', '-ce', 'que', 'ton', 'mari', 'est', 'a...","['-ce', 'mari', 'Boston', '?']",4,10,0,2,0,1,"['est-ce', 'que', 'ton', 'mari', 'est', 'aussi...",6.326896,4.309433,0,"[[0.0041115624, -0.07472146, -0.021084685, 0.0...","[0.0041115624, -0.07472146, -0.021084685, 0.01..."
4,"Dans les écoles de commerce, dans les couloirs...",B1,2,"Dans les écoles de commerce, dans les couloirs...","['Dans', 'les', 'écoles', 'de', 'commerce', ',...","['écoles', 'commerce', ',', 'couloirs', 'place...",24,42,4,10,2,1,"['dans', 'les', 'écoles', 'de', 'commerce,', '...",88.671934,60.137653,1,"[[0.025271175, 0.009502788, -0.07489673, 0.161...","[0.025271175, 0.009502788, -0.07489673, 0.1611..."


In [10]:
pred_features = ['token_count_no_stop', 'cognate_count', 'nb_nouns', 'nb_adj']
X_cam = pd.DataFrame(training['cam_pooled_embedding'].tolist())
X = pd.concat([training[pred_features], X_cam], axis=1)
X.columns = X.columns.astype(str)
y = training[['encoded_diff']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [ ]:
correlation_matrix = training.corr()
correlation_relevant = correlation_matrix['encoded_diff'].abs().sort_values(ascending=False)

<ipython-input-39-d45f1d106043>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = training.corr()


In [ ]:
correlation_relevant

encoded_diff           1.000000
token_count_no_stop    0.529720
nb_nouns               0.516952
token_count            0.497870
nb_adj                 0.461113
nb_verbs               0.384685
tfidf_score_unigram    0.219948
tfidf_score_bigram     0.218584
cognate_count          0.198810
nb_adv                 0.193828
Name: encoded_diff, dtype: float64

In [11]:
from sklearn.linear_model import LogisticRegression
LogReg = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=3000)
LogReg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=3000)

In [12]:
y_pred_RegLog = LogReg.predict(X_test)

In [13]:
# Accuracy on the test set
print('Accuracy of Logistic regression with cognates classifier on test set: {:.2f}'
     .format(LogReg.score(X_test, y_test)))

# Accuracy on the training set
print('Accuracy of Logistic regression with cognates classifier on training set: {:.2f}'
     .format(LogReg.score(X_train, y_train)))

from sklearn.metrics import precision_score, recall_score, f1_score
average_type = 'weighted'

precision = precision_score(y_test, y_pred_RegLog, average=average_type)
recall = recall_score(y_test, y_pred_RegLog, average=average_type)
f1 = f1_score(y_test, y_pred_RegLog, average=average_type)

print('The precision is: {:0.3f}'.format(precision))
print('The recall is: {:0.3f}'.format(recall))
print('The F1 score is: {:0.3f}'.format(f1))

Accuracy of Logistic regression with cognates classifier on test set: 0.57
Accuracy of Logistic regression with cognates classifier on training set: 0.69
The precision is: 0.566
The recall is: 0.568
The F1 score is: 0.567


In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_RegLog)
print(confusion_matrix)

[[125  33   7   1   0   0]
 [ 29  83  41   2   2   1]
 [ 17  44  79  12   7   7]
 [  4   2  17  82  36  12]
 [  0   0   4  29  75  44]
 [  0   1   3  23  37 101]]


In [ ]:
from sklearn.linear_model import LogisticRegression
LogRegFull = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=3000)
LogRegFull.fit(X, y)

In [ ]:
X_cam_pred = pd.DataFrame(test['cam_pooled_embedding'].tolist())
X_pred = pd.concat([test[pred_features], X_cam_pred], axis=1)
X_pred.columns = X_pred.columns.astype(str)
test['difficulty encoded'] = LogRegFull.predict(X_pred)

In [ ]:
#function that cleans and return the final dataset to upload on kaggle
def prep_final_pred(df):
  number_to_level = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

  df['difficulty'] = df['difficulty encoded'].map(number_to_level)
  columns_to_keep = ['id', 'difficulty']
  final_df =  df[columns_to_keep]
  final_df.set_index('id', inplace=True)


  return final_df


In [ ]:
final_test = prep_final_pred(test)

In [ ]:
final_test.to_csv('submission.csv')

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"


100% 8.30k/8.30k [00:01<00:00, 8.35kB/s]
Successfully submitted to Detecting the difficulty level of French texts